In [1]:
from FlightRadar24.api import FlightRadar24API
fr_api = FlightRadar24API()
from time import sleep
from datetime import datetime

In [2]:
import os 
import folium
from folium import plugins
import rioxarray as rxr
import earthpy as et
import earthpy.spatial as es

In [3]:
def checkForAltitudeInterest(flights, min_altitude=48000):
    flights_of_interest = []
    for flight in flights:
        if flight.altitude >= min_altitude:
            flights_of_interest.append(flight)
    return flights_of_interest

In [4]:
def plotInfo(flight, map_, count):
    colors = ['darkred', 'red', 'lightred', 'orange', 'darkpurple', 'purple', 'pink', 'beige', 'darkblue', 'blue', 'cadetblue', 'lightblue', 'darkgreen', 'green', 'lightgreen', 'black', 'gray', 'lightgray']
    data_string = ""
    data_string += ("Call Sign: " + flight.callsign)
    data_string += (" Altitude: " + str(flight.altitude) + " ft")
    data_string += (" Heading: " + str(flight.heading) + "")
    data_string += (" Registration: " + str(flight.registration))
    if flight.aircraft_code == "Q4":
        data_string += (" Aircraft: NORTHROP GRUMMAN RQ-4 Global Hawk")
    else:
        data_string +=  (" Aircraft Code:" + str(flight.aircraft_code))
    folium.Marker(
        location=[flight.latitude, flight.longitude], # coordinates for the marker (Earth Lab at CU Boulder)
        popup=data_string, # pop-up label for the marker
        icon=folium.Icon(color=colors[count % len(colors)],icon="plane"),
    ).add_to(map_)
    return map_

In [ ]:
flights = fr_api.get_flights()
flights = checkForAltitudeInterest(flights)
lat = 0
long = 0
for flight in flights:
    lat = lat + flight.latitude
    long = long + flight.longitude
lat = lat / len(flights)
long = long / len(flights)
last_cordindats = [flights[-1].latitude, flights[-1].longitude]
map_ = folium.Map(location=[lat, long], tiles = 'Stamen Terrain',zoom_start = 2)
count = 0
while True:
    flights = fr_api.get_flights()
    flights = checkForAltitudeInterest(flights)
    
    for flight in flights:
        map_ = plotInfo(flight, map_, count)
    now = datetime.now() # current date and time
    date_time = now.strftime("%m-%d-%Y_%H:%M:%S")
    map_.save("/tf/Incongruity/Flight Tracking/Maps/" + date_time + ".html")
    count += 1
    sleep(30)